In [44]:
import os
import cv2
import csv
import random
import time
import numpy as np
import pandas as pd
import matplotlib.image as mpimg # mpimg 用於讀取圖片
import matplotlib.pyplot as plt # plt 用於顯示圖片
import seaborn as sns

In [45]:
import tensorflow as tf

from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import utils as np_utils
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation, Conv2DTranspose

In [46]:
import h5py
from numpy import loadtxt
from tensorflow.keras.models import load_model
model1= load_model('model1.h5')
model2= load_model('model2.h5')
model3= load_model('model3.h5')
print("載入模型成功!繼續訓練模型")

載入模型成功!繼續訓練模型


放入測試集資料

In [47]:
csvfile_test = open('dev.csv')
reader_test = csv.reader(csvfile_test)

In [48]:
# 讀取csv標籤
labels = []
for line in reader_test:
    tmp = [line[0],line[1]]
    # print tmp
    labels.append(tmp)
csvfile_test.close() 

In [49]:
labels = labels[1:]  # remove the first row
print("芒果圖片數量: ",len(labels))

芒果圖片數量:  800


In [50]:
# 轉換圖片的標籤
for i in range(len(labels)):
    labels[i][1] = labels[i][1].replace("A","0")
    labels[i][1] = labels[i][1].replace("B","1")
    labels[i][1] = labels[i][1].replace("C","2")

In [51]:
x_test = []
y_test = []
image_size = 128
for i in range(len(labels)):
    img = cv2.imread("C1-P1_Dev/" + labels[i][0] )
    res = cv2.resize(img,(image_size,image_size),interpolation=cv2.INTER_LINEAR)
    res = img_to_array(res)
    x_test.append(res)    
    y_test.append(labels[i][1])

In [52]:
y_test_org = y_test

# 轉換至array的格式
x_test = np.array(x_test)
y_test = np.array(y_test)

In [53]:
# 轉換至float的格式
for i in range(len(x_test)):
    x_test[i] = x_test[i].astype('float32')
# x_test = x_test/255

In [54]:
# 標籤進行one-hotencoding
y_test = np_utils.to_categorical(y_test)

開始預測

In [55]:
# 測試集標籤預測
y_pred1 = model1.predict(x_test)
y_pred2 = model2.predict(x_test)
y_pred3 = model3.predict(x_test)

In [56]:
# 模型預測後的標籤
predict_label1 = np.argmax(y_pred1,axis=1)
print(predict_label1)
# print(len(predict_label))
predict_label2 = np.argmax(y_pred2,axis=1)
print(predict_label2)
predict_label3 = np.argmax(y_pred3,axis=1)
print(predict_label3)

[1 0 2 1 2 1 2 1 1 0 0 2 1 1 2 1 0 1 1 1 0 1 2 0 2 0 2 2 2 1 0 0 2 1 0 1 0
 1 0 1 0 2 0 1 1 2 1 0 2 0 1 1 1 2 1 0 1 1 2 0 1 0 2 2 1 2 2 1 2 2 2 1 1 2
 2 2 1 1 2 0 0 2 1 1 2 2 2 2 0 1 2 1 1 2 2 2 1 0 2 1 2 1 1 2 1 1 1 1 1 2 2
 0 2 2 0 2 2 2 1 1 1 2 2 1 1 0 0 2 2 1 1 0 1 1 1 2 2 1 0 2 1 1 2 0 1 1 2 1
 1 1 1 0 1 1 2 0 1 0 2 1 2 1 0 1 2 2 1 2 0 2 1 1 2 2 2 1 0 1 0 1 1 2 1 2 2
 2 0 1 2 1 2 2 1 1 2 1 1 2 1 1 2 1 1 0 1 2 0 1 2 1 2 1 0 2 0 2 1 1 1 1 1 2
 1 1 1 1 0 1 1 1 2 1 1 1 0 1 2 0 2 1 2 2 2 0 1 0 2 2 2 2 1 1 2 2 2 1 1 1 2
 2 2 1 1 1 2 1 2 0 1 1 2 1 1 0 2 2 2 1 2 1 1 0 0 2 2 0 1 2 1 1 2 1 2 1 1 2
 1 1 1 2 2 0 2 0 2 2 1 2 1 2 2 0 2 1 1 2 1 1 1 2 0 1 2 2 2 1 1 1 0 2 1 1 2
 1 2 1 1 2 0 2 1 2 0 2 0 0 0 0 0 1 2 1 1 0 1 1 1 2 1 0 0 1 1 1 2 1 1 1 2 2
 2 1 1 1 1 1 1 2 0 2 0 2 0 1 1 2 1 2 0 2 2 2 1 2 1 0 2 2 1 1 0 0 2 2 1 2 1
 1 2 0 1 2 1 1 1 1 1 2 2 1 1 0 0 2 0 2 1 1 1 2 2 2 1 2 1 2 1 1 1 0 1 1 1 0
 1 1 0 1 1 1 2 2 1 2 0 0 2 0 1 1 2 1 2 1 0 1 1 0 2 1 1 2 2 2 0 1 2 1 2 1 0
 1 1 0 2 1 1 1 2 2 2 2 2 

In [57]:
# 模型原標籤
true_label = y_test_org
true_label = np.array(true_label)
print(true_label)
# print(len(true_label))

['1' '0' '1' '1' '2' '1' '2' '0' '1' '0' '0' '1' '2' '1' '2' '1' '1' '0'
 '0' '1' '0' '0' '2' '0' '1' '0' '2' '2' '1' '1' '1' '0' '2' '2' '0' '0'
 '0' '0' '0' '0' '1' '2' '0' '2' '2' '2' '1' '0' '2' '0' '1' '1' '1' '2'
 '2' '0' '1' '0' '2' '0' '1' '1' '2' '0' '1' '2' '2' '0' '2' '1' '2' '1'
 '1' '2' '2' '2' '1' '0' '2' '0' '0' '2' '1' '1' '2' '2' '2' '2' '0' '1'
 '2' '2' '1' '2' '2' '2' '1' '0' '2' '2' '2' '1' '1' '2' '1' '1' '1' '1'
 '1' '2' '2' '0' '1' '0' '0' '2' '2' '0' '2' '0' '1' '0' '2' '1' '2' '1'
 '0' '1' '2' '1' '0' '0' '0' '1' '0' '2' '1' '1' '0' '2' '0' '1' '2' '0'
 '0' '0' '2' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0' '2' '1' '2' '1'
 '0' '1' '2' '2' '1' '2' '0' '2' '1' '2' '2' '2' '2' '1' '0' '0' '0' '1'
 '2' '2' '0' '2' '1' '2' '0' '0' '2' '1' '2' '2' '1' '1' '2' '1' '0' '2'
 '1' '0' '2' '2' '1' '0' '1' '2' '0' '1' '2' '2' '2' '1' '1' '2' '0' '2'
 '1' '2' '1' '1' '0' '1' '1' '1' '1' '1' '0' '0' '1' '1' '0' '2' '1' '1'
 '0' '1' '2' '0' '2' '0' '1' '2' '2' '0' '1' '0' '2

In [58]:
pd.crosstab(true_label,predict_label1,rownames=['實際值'],colnames=['預測值'])
pd.crosstab(true_label,predict_label2,rownames=['實際值'],colnames=['預測值'])
pd.crosstab(true_label,predict_label3,rownames=['實際值'],colnames=['預測值'])

預測值,0,1,2
實際值,,,
0,160,82,1
1,51,232,10
2,3,126,135


In [59]:
# 整體準確度1
count = 0
for i in range(len(y_pred1)):
    if(np.argmax(y_pred1[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred1)
print('正确率为:%.2f%s' % (score*100,'%'))

正确率为:72.62%


In [60]:
# 整體準確度2
count = 0
for i in range(len(y_pred2)):
    if(np.argmax(y_pred2[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred2)
print('正确率为:%.2f%s' % (score*100,'%'))

正确率为:72.75%


In [61]:
# 整體準確度3
count = 0
for i in range(len(y_pred3)):
    if(np.argmax(y_pred3[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred3)
print('正确率为:%.2f%s' % (score*100,'%'))

正确率为:65.88%


直接比對的話

In [62]:
bagging = predict_label3
for i in range(len(y_pred1)):
    if(np.argmax(y_pred1[i])==np.argmax(y_pred2[i])):
        bagging[i]=np.argmax(y_pred1[i])
    elif(np.argmax(y_pred1[i])==np.argmax(y_pred3[i])):
        bagging[i]=np.argmax(y_pred1[i])
    elif(np.argmax(y_pred2[i])==np.argmax(y_pred3[i])):
        bagging[i]=np.argmax(y_pred2[i])
    else:
        bagging[i]=1
print(bagging)

[1 0 1 1 2 1 2 1 1 0 0 1 1 1 2 1 0 1 0 1 0 1 2 0 2 0 2 2 1 1 0 0 2 2 0 1 0
 1 0 1 0 2 0 1 1 1 1 0 2 0 1 1 1 1 1 0 1 1 2 0 1 0 2 1 1 2 2 1 2 1 2 1 1 2
 2 1 0 1 2 0 0 2 1 0 2 2 2 1 0 1 2 1 1 2 1 2 1 0 2 1 2 1 1 2 1 1 1 0 1 2 0
 0 2 0 0 2 2 1 1 1 1 2 2 1 2 0 0 2 2 1 1 0 1 1 1 2 1 1 0 2 1 1 2 0 1 1 2 1
 1 1 1 0 1 1 1 0 1 0 2 1 2 1 0 1 2 2 0 2 0 2 1 1 2 2 2 1 0 1 0 1 1 2 1 2 0
 2 0 1 2 1 2 2 1 1 2 1 0 2 1 1 2 1 1 0 1 2 0 1 2 1 2 1 0 2 0 2 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 0 1 2 0 2 0 2 2 2 0 1 0 2 1 2 1 1 1 2 2 2 1 1 1 1
 2 2 1 1 1 2 1 2 0 1 1 2 1 1 0 1 2 2 1 2 1 0 0 0 2 2 0 1 2 1 1 2 1 2 1 1 2
 1 1 1 2 2 1 2 0 2 2 1 2 1 2 2 0 2 1 0 1 0 1 1 0 0 1 1 2 2 1 1 1 0 2 1 1 2
 1 2 1 1 2 0 2 1 2 0 2 0 0 0 0 0 1 2 1 1 0 1 1 1 2 1 0 0 1 0 1 2 1 1 1 2 1
 2 2 1 1 1 1 1 2 0 2 0 1 0 1 1 2 1 1 0 1 2 2 1 2 1 0 1 2 1 1 0 0 2 2 0 1 1
 1 1 0 1 1 1 0 1 1 1 2 2 1 1 0 0 2 1 2 1 1 1 2 1 1 1 1 1 2 1 1 1 0 1 1 1 0
 1 0 0 1 0 1 2 2 1 2 0 0 1 0 1 1 2 1 2 2 0 1 1 0 1 1 1 2 2 2 0 1 2 1 1 1 0
 1 1 0 1 1 1 1 2 2 1 1 2 

In [63]:
count = 0
for i in range(len(y_pred3)):
    if(bagging[i] == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred3)
print('正确率为:%.2f%s' % (score*100,'%'))

正确率为:75.12%


用加的

In [64]:
y_pred4 = (y_pred1+y_pred2+y_pred3)/3.0
predict_label4 = np.argmax(y_pred4,axis=1)
print(predict_label4)
pd.crosstab(true_label,predict_label4,rownames=['實際值'],colnames=['預測值'])
count = 0
for i in range(len(y_pred4)):
    if(np.argmax(y_pred4[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred4)
print('正确率为:%.2f%s' % (score*100,'%'))

[1 0 1 1 2 1 2 1 1 0 0 1 1 1 2 1 0 1 0 0 0 1 2 0 1 0 2 2 0 1 0 0 1 2 0 0 0
 0 0 1 0 2 0 1 1 1 1 0 2 0 1 1 1 1 1 0 1 1 2 0 1 0 2 1 1 2 2 0 2 1 2 1 1 2
 2 1 1 0 2 0 0 2 1 1 2 2 2 1 0 1 2 1 1 2 1 2 1 0 2 1 2 1 1 2 1 1 1 0 1 2 2
 0 1 0 0 2 2 1 1 1 1 1 2 1 2 0 0 1 2 1 1 0 1 1 1 2 1 1 0 2 0 1 2 0 1 1 2 1
 1 1 1 0 1 1 1 0 1 0 2 1 2 1 0 1 2 2 0 2 0 2 1 1 2 2 2 1 0 0 0 1 1 2 1 2 0
 2 0 1 2 1 2 2 1 1 2 1 0 2 1 1 2 1 1 0 1 2 0 1 2 1 2 1 0 2 0 2 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 0 1 1 1 0 1 2 0 2 0 2 2 2 0 0 0 2 1 2 1 0 1 2 2 2 0 1 1 1
 2 2 1 1 0 2 1 2 0 1 1 2 1 1 0 0 2 2 1 2 1 0 0 1 2 2 0 1 2 1 1 2 1 2 1 1 1
 0 1 1 2 2 0 2 0 2 2 1 2 1 2 2 0 2 1 0 1 0 1 1 0 0 1 1 2 2 1 1 1 0 2 1 1 2
 1 2 1 1 2 0 2 1 2 0 2 0 0 0 0 0 1 2 1 2 0 0 1 1 2 1 0 0 2 0 1 2 1 1 1 1 1
 2 2 1 1 1 1 1 2 0 2 0 1 0 1 1 2 1 1 0 2 2 2 1 2 1 0 1 2 1 1 0 0 2 1 0 1 1
 1 1 0 1 1 1 0 1 1 1 2 2 1 2 0 0 2 1 2 1 1 1 2 1 1 0 1 1 2 1 1 1 0 1 0 1 0
 1 0 0 1 0 1 2 2 1 2 0 0 1 0 1 1 2 1 2 2 0 1 1 0 1 1 1 2 2 2 0 1 2 1 1 1 0
 1 1 1 1 1 1 1 2 2 1 2 1 